In [2]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
import gzip
import numpy as np
from random import randint
import torch.utils.data as data_utils
from torch.autograd import Variable
from sklearn.metrics.pairwise import cosine_similarity
from numpy import linalg as LA
import torch.nn.utils.rnn as rnn_utils


#torch.manual_seed(1)

In [4]:
#-------------------------------------HELPER FUNCTIONS-------------------------------------#
NUM_NEGATIVE_SAMPLES=20

def N_random_values_in_list(full_list, N):
    x=0
    lower_bound  = 0
    upper_bound = len(full_list)-1
    sample_list=[]
    random_nums=[]
    while x < min(N,len(full_list)):
        random_num = randint(lower_bound, upper_bound) # inclusive range
        if random_num in random_nums:
            continue
        else:
            random_nums.append(random_num)
            x += 1
    return [full_list[i] for i in random_nums]

def convert_to_list(filename):
    if filename.endswith('gz'):
        with gzip.open(filename,'r')as f:
            text_tokens = f.readlines()
    else:
        with open(filename, 'r') as f:
            text_tokens = f.readlines()
    text_tokens = [token.replace('\n','').split('\t') for token in text_tokens]
    text_tokens = [[token[0], token[1].split(' '), token[2].split(' ')] for token in text_tokens]
                   
    return text_tokens

#Sample:question_id, similar_question_id, negative_question_id
def convert_to_samples(filename):
    my_list=convert_to_list(filename)
    new_samples=[]
    for original_sample in my_list:
        for similar in original_sample[1]:
            random_negative_samples = N_random_values_in_list(original_sample[2],NUM_NEGATIVE_SAMPLES)
            new_samples.append([original_sample[0], similar, random_negative_samples])# change this to include all negative 
                                                                                     # examples later
    return new_samples
def make_lookup_table_for_training_data(filename):
    lookup={}
    text_token_list=convert_to_list(filename)
    for token in text_token_list:
        lookup[token[0]] = {'title':token[1],'question':token[2]}
    return lookup
        
#takes  sample_ids of [[q1,p1,n1],[q2,p2,n2]....]
#outputs titles like [[q1_title, p1_title, n1_title],[q2_title,p2_title,n2_title]...]
def convert_sampleids_to_titles(sample_ids,lookup, is_dev_or_test = False):
    #each sample_id [question_id, pos_id, [neg_ids]]
    #print type(sample_ids)==list, "first"
   
    titles = []
   # print "sample_ids", sample_ids
    for sample_id in sample_ids:
        if is_dev_or_test:
            #omit similar questions, only needed for evaluation
            sample_id=[sample_id[0]]+sample_id[2][:]
         #flatten list: [question_id, pos_id, [neg_ids]] --> [question_id, pos_id, neg_id1, neg_id2, ...]
        else:
            sample_id= sample_id[:2]+sample_id[2][:]
        #sample_id : question_id, similar_question_id, negative_question_id
        #try:
        current=[]
        for identity in sample_id:
            try:
                current.append(lookup[str(identity)]['title'])
            except:
                print identity
        titles.append(current)
           # print type(sample_id)==list
        #except:
        #    print sample_id, "is sample id", type(sample_id)==list
    return titles

def remove_non_ascii(text):
    return ''.join([i if ord(i) < 128 else '' for i in text])

def extract_features(word):
    try:
        word=remove_non_ascii(word)
        word=word.encode('utf-8')
    except:
        print(word)
    return word_to_vec.get(word,None)

def find_maximum_title_and_body_length(lookup_table):
    max_len_title = -1
    max_len_question = -1
    max_len_question_id = 0
    for key, dict_val in lookup_table.iteritems():
        len_title = len(dict_val['title'])
        len_question = len(dict_val['question'])
        if len_title > max_len_title:
             max_len_title = len_title
        if len_question > max_len_question:
            max_len_question = len_question
            max_len_question_id = key
    return max_len_title, max_len_question

def title_to_feature_matrix(title_word_list):
    feature_matrix = []
    total_count_of_embeds=0
    for idx, word in enumerate(title_word_list):
        if idx == PARAMETER_MAX_TITLE_LENGTH:
            break
        else:
            word_features = extract_features(word)
            if word_features is not None:
                feature_matrix.append(word_features)
                total_count_of_embeds+=1
        
    #Pad the feature with zeros to ensure all inputs to the net have the same dimension
    feature_matrix += [[0] * NUM_FEATURES_PER_WORD] * (PARAMETER_MAX_TITLE_LENGTH - total_count_of_embeds)
   # print np.array(feature_matrix).T.shape
    
    return np.array(feature_matrix).T



#array is structured like a batch of features 50x200x38
def find_start_of_padding_for_batch(batch):
    vec_lengths_in_batch = []
    for batch_num in range(0, len(batch)):
        single_vec = batch[batch_num]
        length = find_start_of_padding_single_vec(single_vec) + 1
        vec_lengths_in_batch.append(length)
    return vec_lengths_in_batch

#batch = 200x38
def find_start_of_padding_single_vec(single_vec):
    for idx in range(len(single_vec[0])-1, -1, -1):
        if single_vec[0][idx] != 0.:
            return idx
    #if the whole sequence is 0s
    return 0
def create_mask(word_length):
    return np.array([[1. / word_length] * HIDDEN_SIZE] * word_length + [[0] * HIDDEN_SIZE] * (MAX_TITLE_LENGTH - word_length)).T

In [5]:
#-------------------------------------LOAD DATA-------------------------------------#

word_embeddings = 'askubuntu/vector/vectors_pruned.200.txt.gz'
f = gzip.open(word_embeddings, 'r')
wv_text = [ ]
lines = f.readlines()
for line in lines:
    wv_text.append(line.strip())

word_to_vec = {}

for line in wv_text:
    parts = line.split()
    word = parts[0]
    vector = np.array([float(v) for v in parts[1:]])
    word_to_vec[word] = vector
f.close()

In [6]:


#text_tokenized.txt.gz has id \t title \t question body
text_tokenized='askubuntu/text_tokenized.txt.gz'

#train_random.txt
#(1) the query question ID, (2) the list of similar question IDs, and (3) the list of randomly selected question IDs.
train_random_filename='askubuntu/train_random.txt'

#Each line contains (1) the query question ID, (2) the list of similar question IDs, (3) the list of 20 candidate question IDs and (4) the associated BM25 scores of these questions computed by the Lucene search engine. The second field (the set of similar questions) is a subset of the third field.
dev_filename='askubuntu/dev.txt'
test_filename='askubuntu/test.txt'

train_samples = convert_to_samples(train_random_filename)
#dev_samples = convert_to_samples(dev_filename)
#test_samples = convert_to_samples(test_filename)

lookup = make_lookup_table_for_training_data(text_tokenized)
train_titles_only = convert_sampleids_to_titles(train_samples, lookup)

In [7]:
#-------------------------------------NET GLOBAL VARIABLES-------------------------------------#
#-------------------------------------GLOBAL VARIABLES-------------------------------------#
NUM_TRAINING_EXAMPLES = 22850 #FOR DATA BATCHER, WHEN DEPLOYED SHOULD BE ALL TRAINING EXAMPLES
PARAMETER_MAX_TITLE_LENGTH = 38
NUM_FEATURES_PER_WORD = 200 #DO NOT CHANGE. FIXED at 200
MAX_TITLE_LENGTH, MAX_BODY_LENGTH = find_maximum_title_and_body_length(lookup)
KERNEL_SIZE = 3 #MAKE SURE THIS NUMBER IS ODD SO THAT THE PADDING MAKES SENSE
PADDING = (KERNEL_SIZE - 1) / 2
INPUT_SIZE = 200
HIDDEN_SIZE = 150
LEARNING_RATE = 1e-5
MARGIN = 0.2
NUM_EPOCHS = 25
BATCH_SIZE = 50
NUM_BATCHES = NUM_TRAINING_EXAMPLES/BATCH_SIZE

In [16]:
#-------------------------------------CREATE DATA BATCHER-------------------------------------#
#for each tuple of titles make a feature vector that is num_titles x 200 x 38
# where num_titles = 1 (target) + 1 (positive) + n (negative) 


def create_dataset(samples, shuffle_data = True, is_test_or_dev = False):
    features = []
    for sample in samples:
        target_title = sample[0]
        positive_title = sample[1]
        negative_titles = sample[2:]

        target_features = title_to_feature_matrix(target_title)
        positive_features = title_to_feature_matrix(positive_title)
        if len(negative_titles)==0:
            all_features=[target_features, positive_features]
        else:
            n_negative_features = [title_to_feature_matrix(negative_title) for negative_title in negative_titles]
            all_features = [target_features, positive_features] + n_negative_features
        features.append(all_features)            
    targets = torch.LongTensor(len(features)).zero_()
    dataset = data_utils.TensorDataset(torch.FloatTensor(features), targets)
    return dataset

train_data_loaders=[]
for i in range(0,NUM_TRAINING_EXAMPLES,50):
    train_data_loader = create_dataset(train_titles_only[i:i+50])
    print "made training data loader", i
    train_data_loaders.append(train_data_loader)
final_train_dataset=data_utils.ConcatDataset(train_data_loaders)
final_train_data_loader=data_utils.DataLoader(final_train_dataset, batch_size = BATCH_SIZE, shuffle = True)

# features = []
# for i in range(len(train_titles_only[:NUM_TRAINING_EXAMPLES])):   # we should include all but this is just for simplicity 
#     if i%1000 == 0:
#         print i

#     sample = train_titles_only[i]
    
#     target_title = sample[0]
#     positive_title = sample[1]
#     negative_titles = sample[2:]
    
# #     print "\n"
# #     print "Target title: {}".format(" ".join(target_title))
# #     print "Positive title: {}".format(" ".join(positive_title))
# #     for negative in negative_titles:
# #         print "Negative title: {}".format(" ".join(negative))
# #     print "\n"
# #     a
# #     if i > 10:
# #         a
# #     print "target_title: {}".format(target_title)
# #     print "positive title: {}".format(positive_title)
# #     print "negative titles: {}".format(negative_titles)
    
#     target_features = title_to_feature_matrix(target_title)
#     positive_features = title_to_feature_matrix(positive_title)
#     n_negative_features = [title_to_feature_matrix(negative_title) for negative_title in negative_titles]
    
# #     print "Target features shape: {}".format(target_features.shape)
# #     print "Positive features shape: {}".format(positive_features.shape)
# #     print "Negative features[0] shape: {}".format(n_negative_features[0].shape)
# #     print "Num negative features: {}".format(len(n_negative_features)) 
#     all_features = [target_features, positive_features] + n_negative_features
#     features.append(all_features)


# print "Done Loop"
# features = np.array(features) 
# targets = torch.LongTensor(len(features), 1).zero_()
# training_dataset = data_utils.TensorDataset(torch.FloatTensor(features), targets)
# train_loader = data_utils.DataLoader(training_dataset, batch_size = BATCH_SIZE, shuffle = True)
# print "Succesfully made the data batcher"

made training data loader 0
made training data loader 50
made training data loader 100
made training data loader 150
made training data loader 200
made training data loader 250
made training data loader 300
made training data loader 350
made training data loader 400
made training data loader 450
made training data loader 500
made training data loader 550
made training data loader 600
made training data loader 650
made training data loader 700
made training data loader 750
made training data loader 800
made training data loader 850
made training data loader 900
made training data loader 950
made training data loader 1000
made training data loader 1050
made training data loader 1100
made training data loader 1150
made training data loader 1200
made training data loader 1250
made training data loader 1300
made training data loader 1350
made training data loader 1400
made training data loader 1450
made training data loader 1500
made training data loader 1550
made training data loader 1600


made training data loader 13150
made training data loader 13200
made training data loader 13250
made training data loader 13300
made training data loader 13350
made training data loader 13400
made training data loader 13450
made training data loader 13500
made training data loader 13550
made training data loader 13600
made training data loader 13650
made training data loader 13700
made training data loader 13750
made training data loader 13800
made training data loader 13850
made training data loader 13900
made training data loader 13950
made training data loader 14000
made training data loader 14050
made training data loader 14100
made training data loader 14150
made training data loader 14200
made training data loader 14250
made training data loader 14300
made training data loader 14350
made training data loader 14400
made training data loader 14450
made training data loader 14500
made training data loader 14550
made training data loader 14600
made training data loader 14650
made tra

In [60]:
lstm = nn.LSTM(3,3)
inputs = [autograd.Variable(torch.randn(1, 3))
          for _ in range(5)]  # make a sequence of length 5
hidden = (
          autograd.Variable(torch.randn(1, 1, 3)),
          autograd.Variable(torch.randn(1, 1, 3))
         )
for i in inputs:
    out, hidden = lstm(i.view(1, 1, -1), hidden)


In [44]:
class LSTM(nn.Module):
    def __init__(self, embedding_dim, hidden_dim):
        super(LSTM, self).__init__()
        self.hidden_dim=hidden_dim
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.hidden = self.init_hidden()
        
    def init_hidden(self):
        # The axes semantics are (num_layers, minibatch_size, hidden_dim)
        return (autograd.Variable(torch.zeros(BATCH_SIZE, 1, self.hidden_dim)),
                autograd.Variable(torch.zeros(BATCH_SIZE, 1, self.hidden_dim)))

    def forward(self, sequence):
        lstm_out, self.hidden = self.lstm(sequence, self.hidden)
        return self.hidden


In [3]:
# helper class used for computing information retrieval metrics, including MAP / MRR / and Precision @ x


class Evaluation():

    def __init__(self,data):

        self.data = data


    def Precision(self,precision_at):
        scores = []
        for item in self.data:
            temp = item[:precision_at]
            if any(val==1 for val in item):
                scores.append(sum([1 if val==1 else 0 for val in temp])*1.0 / len(temp) if len(temp) > 0 else 0.0)
        return sum(scores)/len(scores) if len(scores) > 0 else 0.0


    def MAP(self):
        scores = []
        missing_MAP = 0
        for item in self.data:
            temp = []
            count = 0.0
            for i,val in enumerate(item):
                if val == 1:
                    count += 1.0
                    temp.append(count/(i+1))
            if len(temp) > 0:
                scores.append(sum(temp) / len(temp))
            else:
                missing_MAP += 1
        return sum(scores)/len(scores) if len(scores) > 0 else 0.0


    def MRR(self):

        scores = []
        for item in self.data:
            for i,val in enumerate(item):
                if val == 1:
                    scores.append(1.0/(i+1))
                    break

        return sum(scores)/len(scores) if len(scores) > 0 else 0.0


In [8]:
cos = nn.CosineSimilarity(dim=1, eps=1e-6)


def evaluate_model_on_test(model, test_data_filename):
    test_data_loader =convert_to_list(test_data_filename)
    
    model.eval() #change model to 'eval' mode
    correct = 0
    total = 0
    p5 = 0
    MRR = 0
    all_data = []
    for sample in test_data_loader:
        #print sample
        target_id=sample[0]
        similar_ids=sample[1]
        #print similar_ids, 'similar'
        candidate_ids=sample[2]
        #print candidate_ids, 'candidates'
        titles = convert_sampleids_to_titles([sample], lookup,is_dev_or_test=True)[0]

        #print titles
        twenty_one_features = [title_to_feature_matrix(title) for title in titles]
        
        cosine_sims = sample_to_cosine_sims(twenty_one_features, model = model)
        #print cosine_sims
        #_, predicted = torch.max(cosine_sims, 0)
       # total += label.size(0)
       # label = label.squeeze()
        
        #correct += ((predicted == label).sum().data.numpy().squeeze())
        cosine_sims = [cos_sim.data.numpy()[0] for cos_sim in cosine_sims]
        #cosine_sims = cosine_sims.data.numpy().T
        #print cosine_sims, "is cos sims"
        #print candidate_ids, 'is candidates'
        gold_labels = [1 if identity in similar_ids else 0 for identity in candidate_ids]
        #print gold_labels, "is gold"
        
        index_order = sorted(range(len(cosine_sims)), reverse = True,key=lambda k: cosine_sims[k])
        #print index_order, "is index order"
        
        data = [gold_labels[index] for index in index_order]
        all_data.append(data)
#         for instance_idx in range(len(cosine_sims)):
#             cosine_sims_at_idx = cosine_sims[instance_idx]            
#             print "Cosine sims at idx: {}".format(cosine_sims_at_idx)
#             a
#             loc = np.squeeze(np.where(cosine_sims_at_idx.argsort()[::-1] == 0))
#             data_instance = [0] * len(cosine_sims_at_idx)
#             data_instance[loc] = 1
#             data.append(data_instance)
        
#         for instance_idx in range(len(cosine_sims)):
#             cosine_sims_at_idx = cosine_sims[instance_idx]
#             top_5_indices = cosine_sims_at_idx.argsort()[-5:][::-1]
#             label_at_idx = 0
#             if label_at_idx in top_5_indices:
#                 p5 += 1
            
#             MRR += 1. / (1. + np.squeeze(np.where(cosine_sims_at_idx.argsort()[::-1] == 0)))
    return all_data 

def sample_to_cosine_sims(sample, model = None):
    #h_0 (num_layers * num_directions, batch, hidden_size): 
    #tensor containing the initial hidden state for each element in the batch.
    h0=autograd.Variable(torch.zeros(1, BATCH_SIZE,HIDDEN_DIM))

    #c_0 (num_layers * num_directions, batch, hidden_size): 
    #tensor containing the initial cell state for each element in the batch.
    c0=autograd.Variable(torch.zeros(1,BATCH_SIZE,HIDDEN_DIM))
    #RE-ORDER DIMENSIONS OF THE SAMPLE
    sample = Variable(torch.FloatTensor(sample))
    #print sample.unsqueeze(0)
    # 1x21x200x38
    sample = sample.unsqueeze(0).permute(1, 0, 2, 3)
    if model != None:
        lstm = model
    #21 x 1 x 200 x 38
    target_question_features = sample[0] # 1 x 1 x 200 x 38
   # print target_question_features.data.numpy()shape, 'Is target"'
    other_question_features = sample[1:] # 20 x 1 x 200 x 38
   # print other_question_features.data.numpy()shape, "is other"

    #Determine lengths to know how many vectors to take the average across.
    target_question_lengths = find_start_of_padding_for_batch(target_question_features.data)
    other_questions_lengths = [find_start_of_padding_for_batch(negative.data) for negative in other_question_features]

    #RUN THROUGH NET
    
#     target_question_lstm_outs, target_question_lstm_hiddens = lstm(target_question_features,(h0,c0)) 
#         positive_question_lstm_outs, positive_question_lstm_hiddens = lstm(positive_question_features,(h0,c0))
#        # 
#         N_negative_question_lstm_output_tuple_list = [lstm(negative,(h0,c0)) for negative in N_negative_question_features]
#         N_negative_question_lstm_outs = [negative[0] for negative in N_negative_question_lstm_output_tuple_list]
#         N_negative_question_lstm_hiddens = [negative[1] for negative in N_negative_question_lstm_output_tuple_list]
        
        
#         #do permutations to make outputs 50 x 200 x 38
#         target_question_lstm_outs=target_question_lstm_outs.permute(1,2,0)
#         positive_question_lstm_outs=positive_question_lstm_outs.permute(1,2,0)
#         N_negative_question_lstm_outs=[negative.permute(1,2,0) for negative in N_negative_question_lstm_outs]
    
    
    
    
    
    target_question_lstm_outs, target_question_lstm_hiddens = lstm(target_question_features.permute(2,0,1), (h0,c0))
    N_other_question_output_tuples = [lstm(other.permute(2,0,1), (h0,c0)) for other in other_question_features]
    N_other_question_lstm_outs=[other[0] for other in N_other_question_output_tuples]
    N_other_question_lstm_hiddens=[other[1] for other in N_other_question_output_tuples]
    
    
    target_question_lstm_outs=target_question_lstm_outs.permute(1,2,0)
    N_other_question_lstm_outs=[other.permute(1,2,0) for other in N_other_question_lstm_outs]
    
    #CREATE MASKS
    target_questions_masks = [create_mask(_) for _ in target_question_lengths] #DIM = 50 x 100 x 38
    other_questions_masks = [[create_mask(length) for length in length_list] #DIM = 50 x 20 x 100 x 38
                                  for length_list in other_questions_lengths]

    #APPLY MASKS
    #Should the multiplicands, the masks, be Float Tensors or Variables? May have to be float tensors to ensure
    #    pytorch's directed graph back-prop is maintained.


    target_question_net_output_masked = target_question_lstm_outs  * Variable(torch.FloatTensor(target_questions_masks))
    other_questions_net_output_masked = [N_other_question_lstm_outs[idx] * 
                                              Variable(torch.FloatTensor(other_questions_masks[idx]))
                                              for idx in range(20)]

    #SUM OVER WORDS
    target_question_net_output_masked_summed = torch.sum(target_question_net_output_masked, dim = 2) #DIM = 50 x 100
    other_questions_net_output_masked_summed = [torch.sum(
                                                    other_questions_net_output_masked[idx], dim = 2
                                                    )for idx in range(20)] #DIM = 20 x 50 x 100


    #cosine_similarity_pos = cos(target_question_net_output_masked_summed, positive_question_net_output_masked_summed)
    # ^ DIM = 50
    cosine_similarities = [cos(target_question_net_output_masked_summed, other_questions_net_output_masked_summed[idx])
                              for idx in range(20)]
    
    # ^ DIM = 20 x 50
    #
    

    #cosine_similarities = torch.stack([cosine_similarity_pos] + cosine_similarities_neg) # DIM = 21 x 50
    return cosine_similarities


In [10]:
EMBEDDING_DIM, HIDDEN_DIM = 200, HIDDEN_SIZE
#lstm = LSTM(EMBEDDING_DIM, HIDDEN_DIM)
#lstm = nn.LSTM(EMBEDDING_DIM, HIDDEN_DIM)
lstm = torch.load("LSTM_Epoch_6")

LEARNING_RATE=1e-5

criterion = nn.MultiMarginLoss(p=1, margin=MARGIN, weight = None, size_average=True) #HAHA just put these in to look smart 
optimizer = torch.optim.Adam(lstm.parameters(), lr = LEARNING_RATE)

cos = nn.CosineSimilarity(dim=1, eps=1e-6)

#h_0 (num_layers * num_directions, batch, hidden_size): 
#tensor containing the initial hidden state for each element in the batch.
h0=autograd.Variable(torch.zeros(1, BATCH_SIZE,HIDDEN_DIM))

#c_0 (num_layers * num_directions, batch, hidden_size): 
#tensor containing the initial cell state for each element in the batch.
c0=autograd.Variable(torch.zeros(1,BATCH_SIZE,HIDDEN_DIM))

for epoch in range(0,NUM_EPOCHS):
    running_loss = 0.0
    for idx,(sample, label) in enumerate(final_train_data_loader):
        print idx, 
        #Sample shape: [50, 22, 200, 38]
        #50 - Batch size, 22 - Num questions per data point
        
        #CNN : BATCH X WORD EMBEDDINGS X WORD
        #LSTM : WORD X BATCH X WORD EMBEDDING
        
        sample = Variable(sample, requires_grad = True)
        label = Variable(label, requires_grad = True)

        #RE-ORDER DIMENSIONS OF THE SAMPLE
        sample = sample.permute(1, 0, 2, 3)

        batch_cos_similarities = []
        batch_num = idx
        
        target_question_features = sample[0] # 50 x 200 x 38
        positive_question_features = sample[1] # 50 x 200 x 38
        N_negative_question_features = sample[2:] #20 x 50 x 200 x 38
        
        #Determine lengths to know how many vectors to take the average across.
        target_question_lengths = find_start_of_padding_for_batch(target_question_features.data)
        positive_question_lengths = find_start_of_padding_for_batch(positive_question_features.data)
        N_negative_questions_lengths = [find_start_of_padding_for_batch(negative.data) for negative in N_negative_question_features]
        
        target_questions_masks = [create_mask(_) for _ in target_question_lengths] #DIM = 50 x 100 x 38
        positive_questions_masks = [create_mask(_) for _ in positive_question_lengths] #DIM = 50 x 100 x 38
        N_negative_questions_masks = [[create_mask(length) for length in length_list] #DIM = 50 x 20 x 100 x 38
                                      for length_list in N_negative_questions_lengths]
        
        target_question_features=target_question_features.permute(2,0,1)
        positive_question_features=positive_question_features.permute(2,0,1)
        N_negative_question_features=N_negative_question_features.permute(0,3,1,2)
        

        # outs are #38x50x200 out, hiddens are 1 x 50 x 200
        # each of the 38 outs are the hidden state at time step t (what we want for mean pooling)
        # hiddens is just the last hidden state (don't really need)
        target_question_lstm_outs, target_question_lstm_hiddens = lstm(target_question_features,(h0,c0)) 
        positive_question_lstm_outs, positive_question_lstm_hiddens = lstm(positive_question_features,(h0,c0))
       # 
        N_negative_question_lstm_output_tuple_list = [lstm(negative,(h0,c0)) for negative in N_negative_question_features]
        N_negative_question_lstm_outs = [negative[0] for negative in N_negative_question_lstm_output_tuple_list]
        N_negative_question_lstm_hiddens = [negative[1] for negative in N_negative_question_lstm_output_tuple_list]
        
        
        #do permutations to make outputs 50 x 200 x 38
        target_question_lstm_outs=target_question_lstm_outs.permute(1,2,0)
        positive_question_lstm_outs=positive_question_lstm_outs.permute(1,2,0)
        N_negative_question_lstm_outs=[negative.permute(1,2,0) for negative in N_negative_question_lstm_outs]
        
        #apply mask
        target_question_net_output_masked = target_question_lstm_outs * Variable(torch.FloatTensor(target_questions_masks))
        positive_question_net_output_masked = positive_question_lstm_outs * Variable(torch.FloatTensor(positive_questions_masks))
        N_negative_questions_net_output_masked = [N_negative_question_lstm_outs[idx] * 
                                                  Variable(torch.FloatTensor(N_negative_questions_masks[idx]))
                                                  for idx in range(NUM_NEGATIVE_SAMPLES)]

        #AVG OVER WORDS
        target_question_net_output_masked_avged = torch.mean(target_question_net_output_masked, dim = 2) #DIM = 50 x 100
        positive_question_net_output_masked_avged = torch.mean(positive_question_net_output_masked, dim = 2) #DIM = 50 x 100
        N_negative_questions_net_output_masked_avged = [torch.mean(
                                                        N_negative_questions_net_output_masked[idx], dim = 2
                                                        )for idx in range(NUM_NEGATIVE_SAMPLES)] #DIM = 20 x 50 x 100
        
        cosine_similarity_pos = cos(target_question_net_output_masked_avged, positive_question_net_output_masked_avged)
        # ^ DIM = 50
        cosine_similarities_neg = [cos(target_question_net_output_masked_avged, N_negative_questions_net_output_masked_avged[idx]) for idx in range(NUM_NEGATIVE_SAMPLES)]
        
        # ^ DIM = 20 x 50
        

        cosine_similarities = torch.stack([cosine_similarity_pos] + cosine_similarities_neg) # DIM = 21 x 50

        optimizer.zero_grad()
        
        cosine_similarities = torch.t(cosine_similarities)
        label = torch.squeeze(label)
        loss = criterion(cosine_similarities, label)
        
        loss.backward()
            
        running_loss += loss.data[0]
            
        optimizer.step()
       # if batch_num % 10 == 0:
            #print "Epoch: {}, Batch: {}, Loss: {}".format(epoch, batch_num, loss.data[0])


    torch.save(lstm, "LSTM_Epoch_{}".format(epoch))
    print ""
    print "Loss after epoch " + str(epoch) + " :" + str(running_loss)
    data = evaluate_model_on_test(lstm, 'askubuntu/dev.txt')
    eval = Evaluation(data)
    print "MAP: ",eval.MAP()
    print "MRR: ", eval.MRR()
    print "P@1: ",eval.Precision(1)
    print "P@5: ",eval.Precision(5)
    


NameError: name 'final_train_data_loader' is not defined

In [13]:
lstm = torch.load("LSTM_Epoch_6")
print "ok"
data = evaluate_model_on_test(lstm, 'askubuntu/dev.txt')
eval = Evaluation(data)
print "MAP: ",eval.MAP()
print "MRR: ", eval.MRR()
print "P@1: ",eval.Precision(1)
print "P@5: ",eval.Precision(5)

ok


KeyboardInterrupt: 

In [24]:
data

<module 'torch.utils.data' from '/Users/nicoleobrien/anaconda/envs/python2/lib/python2.7/site-packages/torch/utils/data/__init__.pyc'>

In [93]:
def toy_batch(seed=11, shape=(25, 1000, 123), classes=10):
    batch_size, max_len, features = shape
    np.random.seed(seed)

    # Samples
    bX = np.float32(np.random.uniform(-1, 1, (shape)))
    b_lenX = np.int32(np.linspace(max_len, max_len, batch_size))
    # print('::: Lengths of samples in batch: {}'.format(b_lenX))

    # Targets
    bY = np.int32(np.random.randint(low=0, high=classes - 1, size=batch_size))

    return bX, b_lenX, bY, classes


In [97]:
bX, b_lenX, bY, classes = toy_batch()
batch_size, max_len, features = bX.shape

# PyTorch compatibility: time first, batch second
bX = np.transpose(bX, (1, 0, 2))

# Create symbolic vars
bX = Variable(torch.from_numpy(bX))
bX = rnn_utils.pack_padded_sequence(bX, b_lenX[::-1])  # Pack those sequences for masking, plz


In [144]:
1+2

3